<a href="https://colab.research.google.com/github/onevay/T_Bank_Sirius_Reviews_Classification/blob/main/BERT_LORA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BERT
Уже есть размеченные данные для train для обучения. Также была создана разметка для test, но она нужна исключительно для валидации. Первым делом попробуем обучить классификатор на нескольких полносвязных слоях с извлечением признаков с помощью bert модели. Для русского языка есть несколько хороших реализаций берт, но для начала можно попробовать **DeepPavlov**

In [ ]:
%pip install torch transformers imbalanced-learn peft accelerate

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import pandas as pd
import re
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from tqdm import tqdm
import os

os.environ["WANDB_DISABLED"] = "true"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BERT_NAME = 'DeepPavlov/rubert-base-cased'

In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BERT_NAME = 'DeepPavlov/rubert-base-cased'
categories = [
    "одежда", "нет товара", "украшения и аксессуары",
    "товары для детей", "текстиль"
]

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/train_labeled.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')
test_labels = pd.read_csv('/content/drive/MyDrive/test_labeled.csv')
test_data.index.name, test_labels.index.name = 'index', 'index'

#test разделен на метки и текстовые данные, нужно объединить
train_data = train_data[train_data['predicted_category'].isin(categories)].copy()
test_data = test_data.merge(test_labels, on='index')
test_data = test_data[test_data['predicted_category'].isin(categories)].copy()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BERT_NAME)

#спецсимволы не помогут классификатору, поэтому удаляем их
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^а-яёa-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_data['text_clean'] = train_data['text'].apply(preprocess_text)
test_data['text_clean'] = test_data['text'].apply(preprocess_text)

label_encoder = LabelEncoder()
label_encoder.fit(categories)
train_data['label'] = label_encoder.transform(train_data['predicted_category'])
test_data['label'] = label_encoder.transform(test_data['predicted_category'])

In [ ]:
class_counts = Counter(train_data['label'])
class_weights = torch.tensor([1.0 / class_counts[i] for i in range(len(categories))], device=device)
class_weights = class_weights / class_weights.sum() * len(categories)

In [ ]:
class AdvancedTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        text = self.texts[idx]

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        #в датасете сразу будем отдавать токены и информацию об объекта (токен, маску и метку)
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
        return item

    def __len__(self):
        return len(self.texts)

Так как для валидации авторазметки LLM моделью мной использовалась ручная валидация выборки в 100 объектов, я заметил сильный дисбаланс классов, поэтому обучить классификатор качественно получится лишь на некоторых категориях, в которых возможно достать нужную часть данных. А для правильной выборки без перевеса в сторону `одежды` использую **RandomOverSampler**

In [ ]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(
    np.array(train_data['text_clean']).reshape(-1, 1),
    np.array(train_data['label'])
)

train_dataset = AdvancedTextDataset(
    X_resampled.flatten().tolist(),
    y_resampled.tolist(),
    tokenizer
)

test_dataset = AdvancedTextDataset(
    test_data['text_clean'].tolist(),
    test_data['label'].tolist(),
    tokenizer
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

In [ ]:
#доп методом от дисбаласа будет кастомная ошибка
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = torch.nn.functional.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

По стандарту в классификаторе ставим полносвязный слой и слой выхода, после проб определился уровень, необходимый для dropout

In [ ]:
base_model = AutoModel.from_pretrained(BERT_NAME).to(device)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION
)

model = get_peft_model(base_model, lora_config).to(device)

classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.2),
    torch.nn.Linear(model.config.hidden_size, 256),
    torch.nn.LayerNorm(256),
    torch.nn.GELU(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(256, len(categories))
).to(device)

loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights).to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def forward_pass(input_ids, attention_mask, labels=None):
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    pooled_output = outputs.last_hidden_state.mean(dim=1)
    logits = classifier(pooled_output)

    loss = None
    if labels is not None:
        loss = loss_fn(logits, labels)

    return (loss, logits) if loss is not None else logits

Для **BERT** возьмем коэффициент для шага, чтобы не влиять сильно на смещение в пользу отдельных классов при создании признаков классификатору

In [ ]:
optimizer = torch.optim.AdamW(
    [
        {"params": model.parameters(), "lr": 5e-5},
        {"params": classifier.parameters(), "lr": 1e-4}
    ],
    weight_decay=0.001
)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=[5e-5, 1e-4],
    steps_per_epoch=len(train_loader),
    epochs=10,
    pct_start=0.1
)

In [ ]:
#можно увеличить количество эпох, но дальше улучшения будут гораздо медленнее
num_epochs = 10
best_f1 = 0
patience = 3
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    classifier.train()
    total_loss = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        loss, logits = forward_pass(input_ids, attention_mask, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(classifier.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})

    avg_loss = total_loss / len(train_loader)

    model.eval()
    classifier.eval()
    test_preds = []
    test_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = forward_pass(input_ids, attention_mask)
            preds = torch.argmax(logits, dim=1)

            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    f1 = f1_score(test_labels, test_preds, average='weighted')
    accuracy = accuracy_score(test_labels, test_preds)

    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), 'best_model_lora.pth')
        torch.save(classifier.state_dict(), 'best_classifier.pth')
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

Validation: 100%|██████████| 895/895 [00:59<00:00, 15.08it/s]


Epoch 1 - Loss: 0.5747, F1: 0.1736, Accuracy: 0.2119


Validation: 100%|██████████| 895/895 [00:59<00:00, 15.07it/s]


Epoch 2 - Loss: 0.0481, F1: 0.6206, Accuracy: 0.5841


Validation: 100%|██████████| 895/895 [00:59<00:00, 15.07it/s]


...

Epoch 8 - Loss: 0.0044, F1: 0.7891, Accuracy: 0.7825


Validation: 100%|██████████| 895/895 [00:59<00:00, 15.13it/s]


Epoch 9 - Loss: 0.0035, F1: 0.7893, Accuracy: 0.7839


Validation: 100%|██████████| 895/895 [01:01<00:00, 14.67it/s]


Epoch 10 - Loss: 0.0035, F1: 0.7905, Accuracy: 0.7842


##Сохранение результатов

In [ ]:
model.load_state_dict(torch.load('best_model_lora.pth'))
classifier.load_state_dict(torch.load('best_classifier.pth'))
model.eval()
classifier.eval()

final_preds = []
final_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Final Evaluation"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        logits = forward_pass(input_ids, attention_mask)
        preds = torch.argmax(logits, dim=1)

        final_preds.extend(preds.cpu().numpy())
        final_labels.extend(labels.cpu().numpy())

test_data['predicted'] = label_encoder.inverse_transform(final_preds)
test_data.reset_index()[['index', 'predicted']].to_csv('test_predictions_lora.csv', index=False)

torch.save(model.state_dict(), 'final_model_lora_weights.pth')
torch.save(classifier.state_dict(), 'final_classifier_weights.pth')

print("Test predictions saved to test_predictions_lora.csv")
print("Model weights saved to final_model_lora_weights.pth and final_classifier_weights.pth")

Final Evaluation: 100%|██████████| 895/895 [01:01<00:00, 14.46it/s]


Test predictions saved to test_predictions_lora.csv
Model weights saved to final_model_lora_weights.pth and final_classifier_weights.pth


По результатам видно, что модель иногда путает детские товары и одижду, отсутствие товара и одежду. Это очевидно, ведь этих классов больше всего, а в товарах для детей пояляется и одежда, как, например, в одежде описывается текстиль. По итогу на **LORA** дообучении с 2-мя слоями nn удалось достичь `f1 = 0.79`, что неплохо, учитывая, что мы сильно не прорабатывали модель, а лишь поработали с дисбалансом классов

In [ ]:
print("Final Test Results:")
print(classification_report(final_labels, final_preds, target_names=categories))
print(f"Weighted F1 Score: {f1_score(final_labels, final_preds, average='weighted'):.4f}")
print(f"Accuracy: {accuracy_score(final_labels, final_preds):.4f}")

conf_matrix = pd.crosstab(
    label_encoder.inverse_transform(final_labels),
    label_encoder.inverse_transform(final_preds),
    rownames=['Actual'],
    colnames=['Predicted']
)
print("\nConfusion Matrix:")
print(conf_matrix)

Final Test Results:
                        precision    recall  f1-score   support

                одежда       0.72      0.85      0.78      1730
            нет товара       0.89      0.80      0.85      4794
украшения и аксессуары       0.39      0.48      0.43       526
      товары для детей       0.00      0.00      0.00        11
              текстиль       0.30      0.47      0.37        97

              accuracy                           0.78      7158
             macro avg       0.46      0.52      0.48      7158
          weighted avg       0.80      0.78      0.79      7158

Weighted F1 Score: 0.7905
Accuracy: 0.7842

Confusion Matrix:
Predicted               нет товара  одежда  текстиль  товары для детей  \
Actual                                                                   
нет товара                    1465     221        33                 2   
одежда                         515    3851       337                 5   
текстиль                        62     202 